In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Reading csv file
df = pd.read_csv('../Datasets/data.csv')

In [ ]:
# Size of dataset (rows, columns)
df.shape

In [ ]:
# Columns
df.columns

In [ ]:
# Datatypes of columns
df.dtypes

In [ ]:
# Checking for null values
df.isnull().sum()

In [ ]:
# Dropping unnamed columns
df = df.iloc[:, :172]

In [ ]:
# Dropping position and time columns
for i in range(1, 43):
    pos = 'Q' + str(i) + 'I'
    time = 'Q' + str(i) + 'E'
    df.drop([pos, time], axis=1, inplace=True)

In [ ]:
# Dropping unnecessary columns
drop_columns = ['engnat', 'hand', 'religion', 'orientation', 'race', 'voted', 'married', 'major', 'country', 'screensize', 'uniquenetworklocation', 'source', 'introelapse', 'testelapse', 'surveyelapse']
df.drop(drop_columns, axis=1, inplace=True)

In [ ]:
# Dropping remaining questions and reordering according to DASS21 scale
Q_to_drop = ['1', '5', '7', '9', '11', '12', '15', '16', '18', '19', '21', '22', '23', '24', '26', '27', '30', '34', '35', '36', '37']
for i in Q_to_drop:
    d = 'Q' + i + 'A'
    df.drop([d], axis=1, inplace=True)